<a href="https://colab.research.google.com/github/simreaney/CAMELS-GB-2-HBV-Light/blob/main/CatchmentControlsOnDischarge_International.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Catchment Controls on River Discharge - International Edition
## Managing River Catchments, Durham University.  
Sim M. Reaney 20234  
In this notebook, you can look at the relationships between the catchment characteristics and the discharge measured from the gauging station. You are going to build a statistical model to predict the runoff ratio based on the catchment descriptors.  You are going to do this analysis for the GB dataset. There are also the USA and Australian datasets available to see if the same relationships hold for a different geographical region.
  
## What you need to do:
* **Mapping patterns** - here you can get a visual sense of the spatial patterns of the controls on runoff for the country you are looking at. this section is for context.
* **Single linear regression** - here you are going to look at each control on the runoff ratio and make a short list of the ones that have the greatest predictive power.
* **Multiple linear regression** - here you will take you short list from the previous step and use it to make a selection of the top three controls whicj when combined, gives the best prediction of the runoff ratio.
* **Multiple non-linear regression** - here you will extend your analysis from the previous step and use non-linear regression to further improve the prediction of the runoff ratio.

The notebook is made up of a set of blocks of Python code, options on the right and the results shown in the graphs and maps. To run a block of code, click the play button in the top right of the code. This button is the triangle in the circle. You will not need to edit any of the code to make this analysis work.

# Mapping patterns

The relationships between the catchment characteristics and the runoff percentage can be considered in a spatial context. Take a bit of time to familiarise yourself with the data and its spatial structure.  
  
In the tool below, you can quickly plot the spatial pattern of the different controls. You can select different colour ramps to show the different controls and you can vary the point size.

The catchment descriptors in this workbook are a sub-set from the [CAMELS-GB](https://essd.copernicus.org/articles/12/2459/2020/) dataset (Coxon et al. 2020), the [CAMELS](https://gdex.ucar.edu/dataset/camels.html) dataset (Newman et al. 2014) for the USA,  the [CAMELS-AUS](https://essd.copernicus.org/articles/13/3847/2021/) dataset for Australia (Fowler et al. 2021) and from [CAMELS-Brazil](https://essd.copernicus.org/articles/12/2075/2020/) (Chagas et al 2020). You can investigate the role and effect of the following descriptors:
* *area_gages2* - catchment area, km<sup>2</sup>
* *elev_mean* - gauge elevation, masl
* *slope_mean* - catchment mean drainage path slope, m km<sup>−1</sup>
* *soil_conductivity* - saturated hydraulic conductivity, cm hr<sup>-1</sup>
* *soil_depth_pelletier* - depth to bedrock (maximum 50 m), m
* *soil_porosity* - volumetric porosity (saturated volumetric water content, %). Only for USA and GB.
* *frac_forest* - percentage cover of deciduous woodland, %
* *p_mean* - mean daily precipitation, mm day<sup>-1</sup>
* *frac_snow* - fraction of the precipitation delivered as snow, %
* *aridity* - aridity, calculated as the ratio of mean daily potential evapotranspiration to mean daily precipitation, index

In this workshop, you are going to focus on the runoff ratio (runoff_ratio). This is the total discharge (m<sup>3</sup>) divided by the total rainfall (m<sup>3</sup>), 0 - 1 (1 = 100%)

First, we need to install a new library called [Contextily](https://contextily.readthedocs.io/) that does the background contexty mapping for the dataset. Once the step below has been completed, then you can create the maps.





In [ ]:
#@title Install and import the libraries - run this block once.
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.plotting import figure, output_file, show, output_notebook

from google.colab import data_table
%load_ext google.colab.data_table

print ("Installing contextily...")
!pip install contextily  --quiet
print ("done")
import contextily as cx

output_notebook()
print("Now, you can use the mapping tool below.")


In [ ]:
#@title Plotting discharge controls on a map
control3 = "soil_conductivity" #@param ["runoff_ratio", "q_mean", "area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'frac_snow','p_mean', 'aridity']
pointSize = 69 #@param {type:"slider", min:5, max:300, step:1}
colourRamp = "viridis" #@param ["autumn", "viridis","plasma","inferno","magma"]
location ="USA" #@param ["GB","USA","AUS","Brazil"]


#Read the CAMELS-GB dataset from the web
df = pd.read_csv('https://github.com/simreaney/simreaney.github.io/raw/master/MRC/workshop/CAMELS-' + location + '.csv', na_values=['?'])
flow = "runoff_ratio"

# #plotting
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.gauge_lon, df.gauge_lat), crs="EPSG:4326")

if location=="GB":
  gdf_bng = gdf.to_crs("EPSG:27700")

  fig, axs = plt.subplots(1, 1, figsize=(10, 10))

  # First map
  axs.set_title(control3)
  gdf_bng.plot(ax=axs, markersize=pointSize, alpha=0.6, column=df[control3], cmap=colourRamp, legend=True)
  cx.add_basemap(axs, crs=gdf_bng.crs, source=cx.providers.OpenStreetMap.Mapnik)

elif location=="USA":

  fig, axs = plt.subplots(1, 1, figsize=(10, 5))

  # First map
  axs.set_title(control3)
  gdf.plot(ax=axs, markersize=pointSize, alpha=0.6, column=df[control3], cmap=colourRamp, legend=True)
  cx.add_basemap(axs, crs=gdf.crs, source=cx.providers.OpenStreetMap.Mapnik)

elif location=="AUS":

  fig, axs = plt.subplots(1, 1, figsize=(10, 5))

  # First map
  axs.set_title(control3)
  gdf.plot(ax=axs, markersize=pointSize, alpha=0.6, column=df[control3], cmap=colourRamp, legend=True)
  cx.add_basemap(axs, crs=gdf.crs, source=cx.providers.OpenStreetMap.Mapnik)

elif location=="Brazil":

  fig, axs = plt.subplots(1, 1, figsize=(10, 5))

  # First map
  axs.set_title(control3)
  gdf.plot(ax=axs, markersize=pointSize, alpha=0.6, column=df[control3], cmap=colourRamp, legend=True)
  cx.add_basemap(axs, crs=gdf.crs, source=cx.providers.OpenStreetMap.Mapnik)


## Single Linear Regression
In this section, you are going to look at the statistical power of the individual catchment descriptors using linear regression. For each of the controls, run the code and note down the R<sup>2</sup> value. You are then going to use this list to inform the multiple regression in the next step.

In [ ]:
control4 = "aridity" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'lai_max', 'frac_snow','p_mean', 'aridity']

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
import numpy as np

reg = linear_model.LinearRegression()
reg.fit(df[[control4]], df['runoff_ratio'])
runoffPred = reg.predict(df[[control4]])

# The coefficients
print("Coefficients: ", reg.coef_)
print("Intercept: ", reg.intercept_)
# The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(df['runoff_ratio'], runoffPred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination (R²): %.2f" % r2_score(df['runoff_ratio'], runoffPred))

# Plot outputs
p1 = figure(y_axis_label='observed runoff ratio', x_axis_label=control4, width=400, height=400)
p1.scatter(df[control4], df['runoff_ratio'], size=2, color='purple', alpha=0.7, legend_label='Individual catchments')
# # Create a line of best fit
x_range = np.linspace(min(df[control4]), max(df[control4]), 100)
y_range = reg.predict(x_range.reshape(-1, 1))
p1.line(x_range, y_range, line_width=2, color='blue', legend_label='Line of Best Fit')

# Plot outputs
p = figure(y_axis_label='predicted runoff ratio', x_axis_label='observed runoff ratio', width=400, height=400)
p.scatter(df['runoff_ratio'], runoffPred, size=2, color='orange', alpha=0.7, legend_label='Individual catchments')

# Create and show a grid of the two plots
grid = gridplot([[p1, p]])
show(grid)


## Multiple Linear Regression
As you will have seen, there is not one individual varable that effectively predicts the runoff ratio. Therefore, in this next step, you are going to do a multiple linear regression analysis using the top set of individual factors from the previous step. It is likely that the top performing model will not be made up simply of the top three R<sup>2</sup> controls from the previous step. You need to consider the information content in each and what it is telling you about the catchment's response to rainfall.

In [ ]:
control_1 = "aridity" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'lai_max', 'frac_snow','p_mean', 'aridity']
control_2 = "slope_mean" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'lai_max', 'frac_snow','p_mean', 'aridity']
control_3 = "elev_mean" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'lai_max', 'frac_snow','p_mean', 'aridity']


regMulti = linear_model.LinearRegression()
regMulti.fit(df[[control_1, control_2, control_3]], df['runoff_ratio'])
runoffPred = regMulti.predict(df[[control_1, control_2, control_3]])

# The coefficients
print("Coefficients: \n", regMulti.coef_)
# The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(df['runoff_ratio'], runoffPred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination (R²): %.2f" % r2_score(df['runoff_ratio'], runoffPred))

# Plot outputs
p = figure(y_axis_label='predicted runoff ratio', x_axis_label='observed runoff ratio', width=600, height=600)
p.scatter(df['runoff_ratio'], runoffPred, size=2, color='purple', legend_label='Individual catchments')

# Add a line of best fit 1:1
x_range = (min(df['runoff_ratio']), max(df['runoff_ratio']))
y_range = (min(runoffPred), max(runoffPred))
p.line(x_range, x_range, line_width=2, line_color="blue", alpha=0.7, legend_label='Line of Best Fit')

show(p)


## Multiple Non-linear regression
You will have seen that many of the relationships between the runoff ratio and the catchment descriptors are non-linear. Therefore, we can explore the data with a regression model that can capture these non-linear relationships. We are going to use a polynomial equation that has the form:  

$$y = ß_0 + ß_1x + ß_2x^2 + … + ß_nx^n$$

Where:
* $y$ is the variable we want to predict,
* $x$ is the input control, such as aridity
* $ß_0$ is the $y$ intercept,
* The other $ß$s are the other coefficients
* $n$ is the degree of the polynomial (the higher n is, the more complex curved lines are created).

You can follow the same steps as for multiple linear regression and work out the three controls that are best able to fit the observed patterns of the runoff ratio. There is a control for adjusting the complexity of the statistical model ('degree'). Ideally, a simpler model is preferred:
> "a model should be as simple as possible but no simpler", [Occam's razor](https://en.wikipedia.org/wiki/Occam%27s_razor)).

Therefore, try to balance the complexity of the model with the predictive power.



In [ ]:
control_1 = "elev_mean" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'frac_snow','p_mean', 'aridity']
control_2 = "slope_mean" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'frac_snow','p_mean', 'aridity']
control_3 = "soil_depth_pelletier" #@param ["area_gages2","elev_mean",'slope_mean','soil_conductivity','soil_depth_pelletier','soil_porosity','frac_forest', 'frac_snow','p_mean', 'aridity']
degree = 2 #@param {type:"slider", min:1, max:6, step:1}

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

poly_reg_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
poly_reg_model.fit(df[[control_1, control_2, control_3]], df['runoff_ratio'])
runoffPred = poly_reg_model.predict(df[[control_1, control_2, control_3]])

# The coefficients
# print("Coefficients: \n", poly_reg_model.named_steps['linearregression'].coef_)
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination (R²): %.2f" % r2_score(df['runoff_ratio'], runoffPred))

# Plot outputs
p = figure(y_axis_label='predicted runoff ratio', x_axis_label='observed runoff ratio', width=600, height=600)
p.scatter(df['runoff_ratio'], runoffPred, size=2, color='purple', legend_label='Individual catchments')

# Add a line of best fit 1:1
x_range = (min(df['runoff_ratio']), max(df['runoff_ratio']))
y_range = (min(runoffPred), max(runoffPred))
p.line(x_range, x_range, line_width=2, line_color="blue", alpha=0.7, legend_label='Line of Best Fit')

show(p)

# Show me the data!
If you want to see the data set that sits behind this analysis, run the block of code below

In [ ]:
# @title
df


## References:  
Chagas, V. B. P., Chaffe, P. L. B., Addor, N., Fan, F. M., Fleischmann, A. S., Paiva, R. C. D., and Siqueira, V. A.: CAMELS-BR: hydrometeorological time series and landscape attributes for 897 catchments in Brazil, Earth Syst. Sci. Data, 12, 2075–2096, https://doi.org/10.5194/essd-12-2075-2020, 2020.

Coxon, G., Addor, N., Bloomfield, J. P., Freer, J., Fry, M., Hannaford, J., Howden, N. J. K., Lane, R., Lewis, M., Robinson, E. L., Wagener, T., and Woods, R. 2020: CAMELS-GB: hydrometeorological time series and landscape attributes for 671 catchments in Great Britain, Earth Syst. Sci. Data, 12, 2459–2483, https://doi.org/10.5194/essd-12-2459-2020

Fowler, K. J. A., Acharya, S. C., Addor, N., Chou, C., and Peel, M. C.: CAMELS-AUS: hydrometeorological time series and landscape attributes for 222 catchments in Australia, Earth Syst. Sci. Data, 13, 3847–3867, https://doi.org/10.5194/essd-13-3847-2021, 2021.

Newman A.; Sampson K., Clark M. P., Bock A., Viger R. J. and Blodgett D., 2014. A large-sample watershed-scale hydrometeorological dataset for the contiguous USA. Boulder, CO: UCAR/NCAR. https://dx.doi.org/10.5065/D6MW2F4D

---

Built with [Pandas](https://pandas.pydata.org/), [Numpy](https://numpy.org/), [GeoPandas](https://geopandas.org/en/stable/), [Bokeh](http://bokeh.org/), [Contextily](https://contextily.readthedocs.io/) and [scikit-learn](https://scikit-learn.org/stable/).

---
Sim M. Reaney, Durham University.  
[simreaney.github.io](https://simreaney.github.io)  
[@simreaney](https://twitter.com/simreaney)